In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/Users/herrakaava/Documents/ML_DATA/Advertising.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [4]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [5]:
X = df.drop('sales', axis=1)
y = df['sales']

<h3>Finding the parameter vector $\, \boldsymbol{\theta} \,$ with sklearn</h3>

In [6]:
pipe = make_pipeline(StandardScaler(), LinearRegression(fit_intercept=True))
pipe.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [7]:
lin_reg = pipe.named_steps['linearregression']

In [8]:
pd.DataFrame({
    'coef': np.concatenate([[lin_reg.intercept_], lin_reg.coef_], axis=0)
}, index=['intercept'] + X.columns.tolist())

,coef
intercept,14.022500
TV,3.919254
radio,2.792063
newspaper,-0.022539


<h3>Finding the parameter vector $\, \boldsymbol{\theta} \,$ with Batch Gradient Descent</h3>

With linear regression, the objective function one aims to minimize (usually) is

$$ \mathcal{J}(\boldsymbol{\theta}) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 = \frac{1}{n} (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta})^T \, (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta}). $$


However, to find the OLS solution (the parameter vector $\, \boldsymbol{\theta}) \,$ that minimize the objective function, we don't need to consider the constant $\, \frac{1}{n}, \,$ as it does not depend on $\, \boldsymbol{\theta}, \,$ and therefore it does not affect the location of the minimum of $\, \boldsymbol{\theta}. \,$


To minimize the objective function, one needs to find the gradient of the objective function $\, \mathcal{J}(\boldsymbol{\theta}) \,$ with respect to the parameter vector $\, \boldsymbol{\theta}. \,$

$$ \frac{\partial}{\partial \boldsymbol{\theta}} \mathcal{J}(\boldsymbol{\theta}) = \begin{bmatrix}
\frac{\partial}{\partial \boldsymbol{\theta}_0} \mathcal{J}(\boldsymbol{\theta}) \\
\frac{\partial}{\partial \boldsymbol{\theta}_1} \mathcal{J}(\boldsymbol{\theta}) \\
\vdots \\
\frac{\partial}{\partial \boldsymbol{\theta}_p} \mathcal{J}(\boldsymbol{\theta}) \\
\end{bmatrix}. $$

The derivation is given below:

\begin{align*}
    (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta})^T \, (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta}) &= (\boldsymbol{y}^T - \boldsymbol{\beta}^T \boldsymbol{X}^T) \, (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta}) \\
    &= \boldsymbol{y}^T \boldsymbol{y} - \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} - \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{y} + \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\beta} \\
    &= \boldsymbol{y}^T \boldsymbol{y} - \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} - \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} + \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\beta} \\
    &= \boldsymbol{y}^T \boldsymbol{y} - 2 \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} + \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\beta}
\end{align*}

Some derivation rules:

$$ \frac{\partial}{\partial \boldsymbol{x}} \boldsymbol{x}^T \boldsymbol{a} = \frac{\partial}{\partial \boldsymbol{x}} \boldsymbol{a}^T \boldsymbol{x} = \boldsymbol{a}. $$

$$ \frac{\partial}{\partial \boldsymbol{x}} \boldsymbol{x}^T \boldsymbol{A} \boldsymbol{x} = (\boldsymbol{A} + \boldsymbol{A}^T) \boldsymbol{x} = 2 \boldsymbol{A} \boldsymbol{x} \quad \text{(if} \, \boldsymbol{A} = \boldsymbol{A}^T). $$

Hence

\begin{equation*}
    \frac{\partial}{\partial \boldsymbol{\theta}} \mathcal{J}(\boldsymbol{\theta}) = -2 \boldsymbol{X}^T \boldsymbol{y} + 2 \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta}
\end{equation*}

And the least squares solution is:

\begin{align*}
    -2 \boldsymbol{X}^T \boldsymbol{y} + 2 \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} &= 0 \\
    2 \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} &= 2 \boldsymbol{X}^T \boldsymbol{y} \\
    \boldsymbol{\theta} &= (\boldsymbol{X}^T \boldsymbol{X})^{-1} \boldsymbol{X}^T \boldsymbol{y}.
\end{align*}

- We will be using the gradient of the objective function with respect to $\, \boldsymbol{\theta} \,$ in the batch gradient descent optimization algorithm. 

- Note that we can actually omit the constant 2 from the gradient descent step as well, as it won't affect the location of the minima of $\, \boldsymbol{\theta}. \,$ 

- Note also that with Batch Gradient Descent, the gradient is calculated over the full training set $\, \boldsymbol{X} \,$ at each GD step (iteration). Hence the name; it uses the whole **batch** of training data et every step.

**Important** note:

- Gradient of a function $\, f \,$ at point $\, x_0 \,$ gives the direction of the fastest **increase**.
- Since we want to go down, we have to reverse the sign to go into the opposite direction (fastest **decrease**).

For the reason mentioned above, a single *Gradient Descent Step* is calculated as:

$$ \boldsymbol{\theta}_{n+1} = \boldsymbol{\theta}_n - \alpha \frac{\partial}{\partial \boldsymbol{\theta}} \mathcal{J}(\boldsymbol{\theta}). $$

Another **important** note:

- When using gradient descent, one should always ensure that all features have a similar scale.

In [58]:
def gradient_descent(X, y, alpha, n_epochs):
    """
    Args:
        X: the feature matrix
        y: the target vector
        alpha: the learning rate
        n_epochs: the number of iterations (gradient descent steps)
    """
    # Randomly initialize the model parameters
    theta = np.random.randn(4,1)
    
    # Gradient descent
    for epoch in range(n_epochs):
        grads = -X.T @ y + X.T @ X @ theta
        theta -= alpha * grads
    
    return theta

In [59]:
def scale_features(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled

In [60]:
# Scale the features
X_scaled = scale_features(X)

# Add a column of constants (for intercept calculation)
X_scaled = sm.add_constant(X_scaled)

In [73]:
theta = gradient_descent(X=X_scaled,
                         y=y.values[:, np.newaxis], 
                         alpha=0.001, 
                         n_epochs=1000)

In [74]:
# Gradient Descent
pd.DataFrame({
    'theta': theta.ravel()
}, index=['intercept'] + X.columns.tolist())

,theta
intercept,14.022500
TV,3.919254
radio,2.792063
newspaper,-0.022539


- Comparing the parameters obtained via gradient descent to the ones obtained with sklearn:

In [75]:
# Sklearn
pd.DataFrame({
    'coef': np.concatenate([[lin_reg.intercept_], lin_reg.coef_], axis=0)
}, index=['intercept'] + X.columns.tolist())

,coef
intercept,14.022500
TV,3.919254
radio,2.792063
newspaper,-0.022539


- As can be seen, the result are the same.